In [1]:
import pandas as pd

#df = pd.read_csv('../data/raw/manual pattern annotations.csv', index_col=0)
df = pd.read_csv('../data/raw/draft - manual annotation-0728.csv', index_col=0)

df.columns = [col.strip() for col in df.columns]
df = df.rename({
    'Participant': 'pid',
    'Task': 'tid',
    'Code snippet': 'snippet',
    'Correntness': 'correctness',
    'Level of investment': 'investment',
    'Rough Direction': 'direction',
    'Code Interaction': 'code_interaction',
    'Vertical Selection Strategy': 'selection_strategy',
    'Intent': 'intent',
    'Time': 'time',
    'Start': 'start_time',
    'Stop': 'end_time',
    
    
},
          axis=1)

df['correctness'] = df['correctness'] > 0

df
# 

,Order per task,snippet,pid,tid,Participant/Task,Task Duration,Understand,correctness,investment,direction,code_interaction,selection_strategy,Horizontal Selection Strategy,time,start_time,end_time,Block Duration,%(Block Duration),intent
All Order,,,,,,,,,,,,,,,,,,,
1,1,Number HRD,52,1,52/1,200,1.0,True,Medium,Backward,Intra procedure,No strategy,NaN,0-7,0,7,7,0.035000,Get the understanding of a part of code block
2,2,Number HRD,52,1,52/1,200,1.0,True,Medium,No direction,Intra procedure,Dynamic,NaN,7-35,7,35,28,0.140000,Get the understanding of a part of code block
3,3,Number HRD,52,1,52/1,200,1.0,True,Medium,Forward,Intra procedure,Linear,NaN,35-55,35,55,20,0.100000,Get the understanding of a part of code block
4,4,Number HRD,52,1,52/1,200,1.0,True,Medium,Forward,Intra procedure,Dynamic,NaN,55-95,55,95,40,0.200000,Get the understanding of a part of code block
5,5,Number HRD,52,1,52/1,200,1.0,True,Quick,Back and forth,Intra procedure,Dynamic - OOF,NaN,95-102,95,102,7,0.035000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,11,Number HRD,48,1,48/1,291,1.0,True,Indepth,Back and forth,Intra procedure,Dynamic,NaN,258-291,258,291,33,0.113402,Validate the hypothesis of PA
689,1,Number HRD,74,3,74/3,500,1.0,True,Medium,Forward,Intra procedure,Linear,Linear Line Oriented,0-23,0,23,23,0.046000,Get the understanding of a part of code block
691,3,Number HRD,74,3,74/3,500,1.0,True,Quick,Back and forth,Intra procedure,Dynamic,NaN,48-147,48,147,99,0.198000,Get a quick overview of the code


In [2]:

df = df[['snippet', 'pid', 'tid', 'correctness', 'investment', 'selection_strategy',
         'direction', 'code_interaction', 'start_time', 'end_time']].copy()


"""
print(df['investment'].unique())
print(df['selection_strategy'].unique())
print(df['direction'].unique())
print(df['code_interaction'].unique())

replacements = {
    'back and forth, generally forward': 'forward',
    'back and forth, generally backward': 'backward',
}
df = df.assign(direction=df['direction'].apply(lambda x: replacements.get(x, x)))
"""
# Replace invalid annotations
valid_pattern_values = {
    "investment": ['Medium' 'Quick' 'Indepth'],
    "selection_strategy": ['No strategy' 'Dynamic' 'Linear'],
    "direction": ['Backward' 'No direction' 'Forward' 'Back and forth'
                  'Reverse the order of definition' 'In the order of definition'
                  'Back and forth the order of denifition'],
    "code_interaction": ['Intra procedure' 'Inter procedure'],
}
for col, valid_values in valid_pattern_values.items():
    df = df.loc[df[col].isin(valid_values)]


# df['pattern'] = list(
#     map(
#         tuple,
#         zip(*(df[col]
#             for col in ['investment', 'selection_strategy', 'direction']))))

df['duration'] = df['end_time'] - df['start_time']
df['pid'] = 'p' + df['pid'].astype(str)
df['tid'] = 't' + df['tid'].astype(str)

df

,snippet,pid,tid,correctness,investment,selection_strategy,direction,code_interaction,start_time,end_time,duration
All Order,,,,,,,,,,,


In [ ]:

df.astype({
    'snippet': 'category',
    'investment': 'category',
    'selection_strategy': 'category',
    'direction': 'category',
    # 'pattern': 'category'
}).set_index(['pid', 'tid']).to_parquet("../data/processed/manual_pattern_annotations.parq")
# df.dtypes


In [7]:
# Re-run the relevant code from the notebook

# Load the dataset
df = pd.read_csv('../data/raw/draft - manual annotation-0728.csv', index_col=0)

# Strip extra spaces from column names
df.columns = [col.strip() for col in df.columns]

# Rename the columns as per the provided mappings
df = df.rename({
    'Participant': 'pid',
    'Task': 'tid',
    'Code snippet': 'snippet',
    'Correntness': 'correctness',
    'Level of investment': 'investment',
    'Rough Direction': 'direction',
    'Code Interaction': 'code_interaction',
    'Vertical Selection Strategy': 'selection_strategy',
    'Time': 'time',
    'Start': 'start_time',
    'Stop': 'end_time',
    'Intent': 'intent',
}, axis=1)

# Convert 'correctness' column values to boolean
df['correctness'] = df['correctness'] > 0

# Select relevant columns and create a copy of the dataframe
df = df[['snippet', 'pid', 'tid', 'correctness', 'investment', 'selection_strategy','direction', 'code_interaction', 'intent', 'start_time', 'end_time']].copy()

# Define replacements
replacements = {
    'back and forth, generally forward': 'forward',
    'back and forth, generally backward': 'backward',
}
df = df.assign(direction=df['direction'].apply(lambda x: replacements.get(x, x)))

# Define valid pattern values
valid_pattern_values = {
    "investment": ['Medium', 'Quick', 'Indepth'],
    "selection_strategy": ['No strategy', 'Dynamic', 'Linear'],
    "direction": ['Backward', 'No direction', 'Forward', 'Back and forth',
                  'Reverse the order of definition', 'In the order of definition',
                  'Back and forth the order of denifition'],
    "code_interaction": ['Intra procedure', 'Inter procedure'],
}

# Filter dataframe based on valid pattern values
for col, valid_values in valid_pattern_values.items():
    df = df.loc[df[col].isin(valid_values)]

# Calculate duration
df['duration'] = df['end_time'] - df['start_time']

# Format 'pid' and 'tid' columns
df['pid'] = 'p' + df['pid'].astype(str)
df['tid'] = 't' + df['tid'].astype(str)

df


,snippet,pid,tid,correctness,investment,selection_strategy,direction,code_interaction,intent,start_time,end_time,duration
All Order,,,,,,,,,,,,
1,Number HRD,p52,t1,True,Medium,No strategy,Backward,Intra procedure,Get the understanding of a part of code block,0,7,7
2,Number HRD,p52,t1,True,Medium,Dynamic,No direction,Intra procedure,Get the understanding of a part of code block,7,35,28
3,Number HRD,p52,t1,True,Medium,Linear,Forward,Intra procedure,Get the understanding of a part of code block,35,55,20
4,Number HRD,p52,t1,True,Medium,Dynamic,Forward,Intra procedure,Get the understanding of a part of code block,55,95,40
6,Number HRD,p52,t1,True,Medium,Linear,Back and forth,Intra procedure,Get the understanding of a part of code block,102,116,14
...,...,...,...,...,...,...,...,...,...,...,...,...
688,Number HRD,p48,t1,True,Indepth,Dynamic,Back and forth,Intra procedure,Validate the hypothesis of PA,258,291,33
689,Number HRD,p74,t3,True,Medium,Linear,Forward,Intra procedure,Get the understanding of a part of code block,0,23,23
691,Number HRD,p74,t3,True,Quick,Dynamic,Back and forth,Intra procedure,Get a quick overview of the code,48,147,99


In [8]:
#save to parq again
df.astype({
    'snippet': 'category',
    'investment': 'category',
    'selection_strategy': 'category',
    'direction': 'category',
    # 'pattern': 'category'
}).set_index(['pid', 'tid']).to_parquet("../data/processed/manual_pattern_annotations.parq")
# df.dtypes
